# MSA 2023 Phase 2 - Run Training Experiment

## 1. Load and connect to workplace

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.52.0 to work with MSA-Phase2-Azure


## 2. Create a parameterised training script

In [3]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation.csv', os.path.join(training_folder, "market_segmentation.csv"))

'training\\market_segmentation.csv'

In [4]:
%%writefile $training_folder/training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

run.complete()

Overwriting training/training.py


## 3. Create a compute cluster

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aguo921"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## 4. Run the script with arguments

In [7]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "training/environment.yaml")

# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True),
                                compute_target=cluster_name) 

# submit the experiment run
experiment_name = 'train-segmentation'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Block until the experiment run has completed
run.wait_for_completion()

{'runId': 'train-segmentation_1690711593_de16741d',
 'target': 'aguo921',
 'status': 'Completed',
 'startTimeUtc': '2023-07-30T10:17:12.778654Z',
 'endTimeUtc': '2023-07-30T10:18:29.353115Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcdsi',
  'ContentSnapshotId': '72625ca4-dbfe-429e-b555-77db63b1093c',
  'azureml.git.repository_uri': 'https://github.com/aguo921/2023-Phase-2.git',
  'mlflow.source.git.repoURL': 'https://github.com/aguo921/2023-Phase-2.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '925700ddc7dfe061c2b5ae022e1171975d4bbea9',
  'mlflow.source.git.commit': '925700ddc7dfe061c2b5ae022e1171975d4bbea9',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg

In [8]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, ":", metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate : 0.01
Accuracy : 0.4900296987696224
AUC A vs rest : 0.7113390862280469
AUC B vs rest : 0.6641015473983684
AUC C vs rest : 0.754124484439445
AUC D vs rest : 0.8668678371578138


azureml-logs/20_image_build_log.txt
outputs/model.pkl
system_logs/cs_capability/cs-capability.log
system_logs/hosttools_capability/hosttools-capability.log
system_logs/lifecycler/execution-wrapper.log
system_logs/lifecycler/lifecycler.log
system_logs/lifecycler/vm-bootstrapper.log
system_logs/metrics_capability/metrics-capability.log
system_logs/snapshot_capability/snapshot-capability.log
user_logs/std_log.txt


## 5. Register the model

In [9]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/model.pkl', model_name='segmentation-model',
                   tags={'Training context':'Script'},
                   properties={
                    'AUC A vs rest': run.get_metrics()['AUC A vs rest'],
                    'AUC B vs rest': run.get_metrics()['AUC B vs rest'],
                    'AUC C vs rest': run.get_metrics()['AUC C vs rest'],
                    'AUC D vs rest': run.get_metrics()['AUC D vs rest'],
                    'Accuracy': run.get_metrics()['Accuracy']
                    })

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

segmentation-model version: 2
	 Training context : Script
	 AUC A vs rest : 0.7113390862280469
	 AUC B vs rest : 0.6641015473983684
	 AUC C vs rest : 0.754124484439445
	 AUC D vs rest : 0.8668678371578138
	 Accuracy : 0.4900296987696224


segmentation-model version: 1
	 Training context : Script
	 AUC A vs rest : 0.7113390862280469
	 AUC B vs rest : 0.6641015473983684
	 AUC C vs rest : 0.754124484439445
	 AUC D vs rest : 0.8668678371578138
	 Accuracy : 0.4900296987696224


segmentation-logistic-regression version: 1


